# libs

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

# Preprocessing

In [3]:
df_modcloth = pd.read_json("modcloth_final_data.json", lines=True)

In [4]:
df_modcloth_clean = df_modcloth.dropna(subset=["user_id","item_id"]).drop_duplicates()

In [5]:
vectorizer = CountVectorizer()
category_bow = vectorizer.fit_transform(df_modcloth_clean['category'])

In [6]:
###############################################
# 3. Implicit Feedback
###############################################

fit_mapping = {'fit': 0, 'large': 2, 'small': 1}
df_modcloth_clean.loc[:, 'fit'] = df_modcloth_clean['fit'].map(fit_mapping)


user_reviews = df_modcloth_clean.groupby('user_id')['review_text'].count().to_dict()
df_modcloth_clean.loc[:, 'num_user_reviews'] = df_modcloth_clean['user_id'].map(user_reviews).fillna(0).astype(int)
max_reviews = df_modcloth_clean['num_user_reviews'].max()
df_modcloth_clean.loc[:, 'num_user_reviews_normalized'] = df_modcloth_clean['num_user_reviews'] / max_reviews

max_quality = df_modcloth_clean['quality'].max()
df_modcloth_clean.loc[:, 'quality_normalized'] = df_modcloth_clean['quality'] / max_quality


df_modcloth_clean.loc[:, 'interaction_strength'] = (
    df_modcloth_clean['fit'] * 0.33 +
    df_modcloth_clean['num_user_reviews'] * 0.33 +
    df_modcloth_clean['quality'] * 0.34
)

In [7]:
df_modcloth_clean['fit'] = df_modcloth_clean['fit'].fillna(df_modcloth_clean['fit'].median())
df_modcloth_clean['num_user_reviews'] = df_modcloth_clean['num_user_reviews'].fillna(df_modcloth_clean['num_user_reviews'].median())
df_modcloth_clean['quality'] = df_modcloth_clean['quality'].fillna(df_modcloth_clean['quality'].median())

<ipython-input-7-939be0d95d69>:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_modcloth_clean['fit'] = df_modcloth_clean['fit'].fillna(df_modcloth_clean['fit'].median())


In [8]:
def calculate_sparsity(df):
    """Calculates sparsity for each numerical feature in the DataFrame.

    Args: dataset as df
    Return : dictionary with item feature name and sparsity values
    """
    numerical_features = df.select_dtypes(include=np.number).columns
    sparsity_results = {}

    for feature in numerical_features:
        sparsity = df[feature].isnull().sum() / len(df)
        sparsity_results[feature] = sparsity

    return sparsity_results

sparsity_dict = calculate_sparsity(df_modcloth_clean)


for feature, sparsity in sparsity_dict.items():
    print(f"Sparsity of {feature}: {sparsity:.2f}")

Sparsity of item_id: 0.00
Sparsity of waist: 0.97
Sparsity of size: 0.00
Sparsity of quality: 0.00
Sparsity of hips: 0.32
Sparsity of bra size: 0.07
Sparsity of fit: 0.00
Sparsity of user_id: 0.00
Sparsity of shoe size: 0.66
Sparsity of num_user_reviews: 0.00
Sparsity of num_user_reviews_normalized: 0.00
Sparsity of quality_normalized: 0.00


In [9]:
def preprocess_data(df, sparsity_threshold=0.50, user_id_col='user_id'):
    """Preprocesses the data by removing numerical features with high sparsity
    Args:
    df : dataset
    sparsity_threshold: how sparse can a feature be to be accepted
    user_id columns

    Return:
    preprocessed dataset -> df_modcloth_clean
    """
    numerical_cols = df.select_dtypes(include=np.number).columns
    cols_to_check = [col for col in numerical_cols if col != user_id_col]

    for col in cols_to_check:
        sparsity = df[col].isnull().sum() / len(df)
        if sparsity > sparsity_threshold:
            print(f"Dropping sparse feature: {col} (Sparsity: {sparsity:.2f})")
            df = df.drop(col, axis=1)

    user_interaction_counts = df_modcloth_clean.groupby('user_id')['interaction_strength'].count()
    print(user_interaction_counts)
    users_with_more_than_two_interactions = user_interaction_counts[user_interaction_counts >= 2]
    print(f"Number of users with more than 2 interactions: {len(users_with_more_than_two_interactions)}")
    df = df_modcloth_clean[df_modcloth_clean["user_id"].isin(users_with_more_than_two_interactions.index)]

    return df

In [10]:
df_modcloth_clean = preprocess_data(df_modcloth_clean, sparsity_threshold=0.50)

Dropping sparse feature: waist (Sparsity: 0.97)
Dropping sparse feature: shoe size (Sparsity: 0.66)
user_id
6         1
46        1
55        1
66        4
104       2
         ..
999864    1
999887    2
999888    1
999923    3
999972    3
Name: interaction_strength, Length: 47958, dtype: int64
Number of users with more than 2 interactions: 15924


In [11]:
df_modcloth_clean.head()

,item_id,waist,size,quality,cup size,hips,bra size,category,bust,height,...,fit,user_id,shoe size,shoe width,review_summary,review_text,num_user_reviews,num_user_reviews_normalized,quality_normalized,interaction_strength
0,123373,29.0,7,5.0,d,38.0,34.0,new,36,5ft 6in,...,1,991571,NaN,NaN,NaN,NaN,2,0.08,1.0,2.69
4,123373,NaN,18,5.0,b,NaN,36.0,new,NaN,5ft 2in,...,1,944840,NaN,NaN,NaN,NaN,1,0.04,1.0,2.36
5,123373,27.0,11,5.0,c,41.0,36.0,new,NaN,5ft 4in,...,1,162012,NaN,NaN,NaN,NaN,1,0.04,1.0,2.36
8,123373,NaN,30,4.0,d,50.0,42.0,new,NaN,5ft 10in,...,1,279568,11.0,wide,NaN,NaN,4,0.16,0.8,3.01
9,123373,NaN,13,5.0,dd/e,41.0,36.0,new,39,5ft 6in,...,0,950172,9.0,NaN,NaN,NaN,2,0.08,1.0,2.36


In [12]:
user_activity = df_modcloth_clean.groupby('user_id')['interaction_strength'].sum()

In [13]:
median_interaction_strength = df_modcloth_clean.groupby('user_id')['interaction_strength'].sum().median()

df_modcloth_clean['user_activity'] = df_modcloth_clean.groupby('user_id')['interaction_strength'].transform('sum').map(
    lambda x: 'Low' if x < median_interaction_strength else 'High'
)

print(df_modcloth_clean['user_activity'].value_counts())

user_activity
High    34900
Low     15471
Name: count, dtype: int64


<ipython-input-13-4ceca4827323>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_modcloth_clean['user_activity'] = df_modcloth_clean.groupby('user_id')['interaction_strength'].transform('sum').map(


# SPLIT AND MATRIX

In [14]:
from sklearn.model_selection import train_test_split

# SPLIT like Exploring data splitting strategies for the evaluation of recommendation models
# - the random split
train_df, temp_df = train_test_split(df_modcloth_clean, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

In [15]:
###############################################
# 3. User Item Matrix
###############################################
# Rows = users.
#Columns = items.
#cell values represent user  interacted with an item.
train_user_item_matrix = train_df.pivot_table(
    index="user_id",
    columns="item_id",
    values="interaction_strength",
    aggfunc="sum",
    fill_value=0
)
val_user_item_matrix = val_df.pivot_table(
    index="user_id",
    columns="item_id",
    values="interaction_strength",
    aggfunc="sum",
    fill_value=0
)
test_user_item_matrix = test_df.pivot_table(
    index="user_id",
    columns="item_id",
    values="interaction_strength",
    aggfunc="sum",
    fill_value=0
)

<ipython-input-15-767f8a9cf3c9>:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_user_item_matrix = train_df.pivot_table(
<ipython-input-15-767f8a9cf3c9>:14: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  val_user_item_matrix = val_df.pivot_table(
<ipython-input-15-767f8a9cf3c9>:21: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_

# Definition ndcg, hr,mrr

In [16]:
###############################################
# 3. Utility/Ranking metrics
###############################################
def dcg_at_k_recursive(relevance_scores, k, b=2):
    """
    Calculate the Discounted Cumulative Gain at rank k recursively.

    :param relevance_scores: A list or array of relevance scores for the ranked items.
    :param k: The rank at which to stop (k items to evaluate).
    :param b: The base of the logarithm (typically 2).
    :return: DCG at rank k.
    """
    k = min(k, len(relevance_scores))
    if k == 0:
        return 0.0

    if k < b:
        return np.sum([rel / np.log2(idx + 2) for idx, rel in enumerate(relevance_scores[:k])])

    else:
        dcg_k_minus_1 = dcg_at_k_recursive(relevance_scores, k - 1, b)
        rel_k = relevance_scores[k - 1]
        return dcg_k_minus_1 + (rel_k / np.log2(k + 1))  #

def dcg_at_k(relevance_scores, k, b=2):
    """
    Calculate the Discounted Cumulative Gain at rank k (non-recursive).
    """
    dcg = 0
    for idx, rel in enumerate(relevance_scores[:k]):
        dcg += rel / np.log2(idx + 2)
    return dcg
def idcg_at_k(relevance_scores, k):
    """
    Calculate the Ideal Discounted Cumulative Gain at rank k.
    :param relevance_scores: A list or array of relevance scores for the ranked items.
    :param k: The rank at which to stop (k items to evaluate).
    :return: Ideal DCG at rank k.
    """
    relevance_scores = sorted(relevance_scores, reverse=True)
    return dcg_at_k(relevance_scores, k)

def ndcg_at_k(relevance_scores, k):
    """
    Calculate the Normalized Discounted Cumulative Gain at rank k.
    :param relevance_scores: A list or array of relevance scores for the ranked items.
    :param k: The rank at which to stop (k items to evaluate).
    :return: nDCG at rank k.
    """
    dcg = dcg_at_k_recursive(relevance_scores, k)
    idcg = idcg_at_k(relevance_scores, k)
    if idcg == 0:
        return 0
    return dcg / idcg


In [17]:
def hr_at_k(actual, predicted, k=5):
    """
    Computes Hit Rate (HR) at rank k.

    Args:
        actual: list of relevant items
        predicted: ranked list of items
        k: rank cutoff (default is 5)

    Returns:
        Hit Rate at rank k.
    """

    for item in predicted[:k]:
        if item in actual:
            return 1
    return 0



def hit_rate_at_k(actual, predicted, k=5):
    """
    Computes Mean Hit Rate (HR) at rank k for all users.

    Args:
        actual: list of relevant items for each user
        predicted: list of predicted ranked items for each user
        k: rank cutoff (default is 5)

    Returns:
        HR at rank k.
    """

    if not actual or len(predicted) == 0:
        return 0.0
    hr_scores = []
    for user_actual, user_predicted in zip(actual, predicted):
        hr = hr_at_k(user_actual, user_predicted, k)
        hr_scores.append(hr)
    return np.mean(hr_scores)

In [18]:
def rr_at_k(actual, predicted, k=5):
    """
    Computes Reciprocal Rank at rank k.

    Args:
        actual: the relevant item(s)
        predicted: the ranked list of items
        k: rank cutoff (default is 5)

    Returns:
        Reciprocal Rank at rank k.
    """

    for i, item in enumerate(predicted[:k]):
        if item == actual:
            return 1 / (i + 1)
    return 0.0


def mrr_at_k(actual, predicted, k=5):
    """
    Computes Mean Reciprocal Rank (MRR) at rank k.

    Args:
        actual: list of relevant items (can be multiple)
        predicted: ranked list of items
        k: rank cutoff (default is 5)

    Returns:
        MRR at rank k.
    """

    if actual is None or len(predicted) == 0:
        return 0.0


    rr_scores = []
    for item in actual:
        rr = rr_at_k(item, predicted, k)
        rr_scores.append(rr)
    return np.mean(rr_scores)



# Definition fairness

In [19]:
###############################################
# 3. Fairness metrics
###############################################
def calculate_disparate_impact(protected_outcomes, privileged_outcomes):
    """
    Args:
        protected_outcomes: List of binary outcomes (1=favorable) for the protected group.
        privileged_outcomes: List of binary outcomes for the privileged group.
    Returns:
        Disparate impact ratio.
    """
    protected_rate = np.mean(protected_outcomes)
    privileged_rate = np.mean(privileged_outcomes)

    if privileged_rate == 0:
        return np.inf

    return protected_rate / privileged_rate


def calculate_group_recommender_unfairness(group1_metrics, group2_metrics):
  """
    Calculates the absolute difference in mean metrics between two groups.
    This metric quantifies the unfairness of a recommender system by examining
    the absolute difference in average performance between different user groups.

    Args:
        group1_metrics (list or numpy.ndarray): A list or numpy array of metrics for group 1.
        group2_metrics (list or numpy.ndarray): A list or numpy array of metrics for group 2.

    Returns:
        float: The absolute difference in mean metrics between the two groups.
  """
  return np.abs(np.mean(group1_metrics) - np.mean(group2_metrics))


In [20]:
def coefficient_of_variation(arr):
    mean_val = np.mean(arr)
    if mean_val == 0:
        return 0
    return np.std(arr) / mean_val

def calculate_ucv(metric_low_group, metric_high_group):

    cv_low = coefficient_of_variation(metric_low_group) if len(metric_low_group) > 0 else 0
    cv_high = coefficient_of_variation(metric_high_group) if len(metric_high_group) > 0 else 0
    return (cv_low + cv_high) / 2

def coefficient_of_variance(group):
    """Calculates the coefficient of variance for a group."""
    return np.std(group) / np.mean(group) if np.mean(group) != 0 else 0

def coef_variation(arr):
    mean_val = np.mean(arr)
    if mean_val == 0:
        return 0
    return np.std(arr) / mean_val



# Content-based

In [21]:
df_modcloth_clean.head()

,item_id,waist,size,quality,cup size,hips,bra size,category,bust,height,...,user_id,shoe size,shoe width,review_summary,review_text,num_user_reviews,num_user_reviews_normalized,quality_normalized,interaction_strength,user_activity
0,123373,29.0,7,5.0,d,38.0,34.0,new,36,5ft 6in,...,991571,NaN,NaN,NaN,NaN,2,0.08,1.0,2.69,High
4,123373,NaN,18,5.0,b,NaN,36.0,new,NaN,5ft 2in,...,944840,NaN,NaN,NaN,NaN,1,0.04,1.0,2.36,Low
5,123373,27.0,11,5.0,c,41.0,36.0,new,NaN,5ft 4in,...,162012,NaN,NaN,NaN,NaN,1,0.04,1.0,2.36,Low
8,123373,NaN,30,4.0,d,50.0,42.0,new,NaN,5ft 10in,...,279568,11.0,wide,NaN,NaN,4,0.16,0.8,3.01,High
9,123373,NaN,13,5.0,dd/e,41.0,36.0,new,39,5ft 6in,...,950172,9.0,NaN,NaN,NaN,2,0.08,1.0,2.36,High


In [22]:

df_modcloth_clean['category'] = df_modcloth_clean['category'].fillna('unknown')


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity




def create_weighted_user_profile(user_id, df, interaction_data):
    """
    calculates weighted user profiles with category and interactions
    Args:
    user_id: users
    df: dataset
    interaction_data: user-item interaction matrix
    :return: weighted user profile
    """
    if user_id not in interaction_data.index:
        return ' '
    user_items = interaction_data.loc[user_id, interaction_data.loc[user_id] > 0].index
    user_items = user_items.tolist()
    user_items_df = df[df['item_id'].isin(user_items)]
    weighted_categories = user_items_df.groupby('category')['interaction_strength'].sum()
    user_profile = ' '.join([f'{category} ' * int(weight) for category, weight in weighted_categories.items()])
    return user_profile

def recommend_items_for_user(user_id, df, interaction_data, top_n=5):
    """
    Recommends items for a user based on category and interaction strength.

    Args:
    user_id : users
    df: dataset
    interaction_data: user-item interaction matrix
    top_n: number of items to recommend

    Returns:
    recommended_items: recommended items for the user
    """
    user_profile = create_weighted_user_profile(user_id, df, interaction_data)
    vectorizer = TfidfVectorizer(stop_words='english')
    all_profiles = df['category'].tolist() + [user_profile]
    profile_matrix = vectorizer.fit_transform(all_profiles)
    cosine_sim = cosine_similarity(profile_matrix[-1:], profile_matrix[:-1])
    sim_scores = list(enumerate(cosine_sim[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    item_indices = [i[0] for i in sim_scores[:top_n]]
    item_scores = [i[1] for i in sim_scores[:top_n]]
    recommended_items = df.iloc[item_indices]

    return recommended_items[['item_id', 'category']], item_scores


recommended_items, scores = recommend_items_for_user(user_id=279568, df=df_modcloth_clean, interaction_data=train_user_item_matrix)
print("Recommended Items for User 123 based on Category and Interaction Strength:")
print(recommended_items)

Recommended Items for User 123 based on Category and Interaction Strength:
       item_id category
38581   422651     tops
38583   422651     tops
38584   422651     tops
38585   422651     tops
38587   422651     tops


In [23]:
def calculate_metrics_for_user(user_id, actual_item, recs, engagement_level):
    """
    calculate all the effecitveness metrics for user
    user_id: user id
    actual_item: actual item the user interacts with
    recs: recommended item for the user
    engagement_level: engagement level of the user
    :return: metrics for the user
    """
    relevance_scores = [1 if item == actual_item else 0 for item in recs]
    ndcg = ndcg_at_k(relevance_scores, k=6)
    hr_val = hr_at_k([actual_item], recs, k=6)
    mrr_val = mrr_at_k([actual_item], recs, k=6)
    cv = coefficient_of_variance(relevance_scores)
    cv_rel = coefficient_of_variance(relevance_scores)
    return [user_id, ndcg, hr_val, mrr_val,cv,cv_rel, engagement_level]

# multiple rounds

In [24]:
def create_user_based_profile(user_id, interaction_data, k=5):
    """
    creates a user profile based on the items interacted with by similar users
    Args:
    user_id: user id
    interaction_data: user-item interaction matrix
    k: number of similar users to consider
    :return the recommended items for the user
    """

    user_interactions = interaction_data.loc[user_id]
    user_similarities = compute_user_similarity(user_interactions, interaction_data)
    similar_users = user_similarities.nlargest(k).index
    recommended_items = interaction_data.loc[similar_users].mean(axis=0).sort_values(ascending=False).index
    recommended_items = [item for item in recommended_items if user_interactions[item] == 0]

    return recommended_items[:k]
from sklearn.metrics.pairwise import cosine_similarity

def compute_user_similarity(target_user_interactions, interaction_data):
    """
    Computes the cosine similarity between a target user and all other users in the interaction data.
    target_user_interactions:
    """
    similarities = cosine_similarity(target_user_interactions.values.reshape(1, -1), interaction_data.values)
    similarity_scores = pd.Series(similarities.flatten(), index=interaction_data.index)

    return similarity_scores
def calculate_metrics_for_user(user_id, actual_item, recommended_items, user_engaged, k=5):
    """
    calculate all the effecitveness metrics for user
    user_id: user id
    actual_item: actual item the user interacts with
    recs: recommended item for the user
    engagement_level: engagement level of the user
    k: number of recommended items
    :return: metrics for the user

    """
    hr = hr_at_k([actual_item], recommended_items, k)

    mrr = mrr_at_k([actual_item], recommended_items, k)

    relevance_scores = [1 if item == actual_item else 0 for item in recommended_items[:k]]
    dcg = sum([rel / np.log2(i + 2) for i, rel in enumerate(relevance_scores)])
    idcg = sum([1 / np.log2(i + 2) for i in range(min(k, len(relevance_scores)))])
    ndcg = dcg / idcg if idcg > 0 else 0
    cv = coefficient_of_variance(recommended_items)
    cv_rel = coefficient_of_variance(relevance_scores)

    return {
        'User': user_id,
        'NDCG@K': ndcg,
        'HR@K': hr,
        'MRR@K': mrr,
        'CV': cv,
        'CV_rel': cv_rel,
        'Engagement Group': user_engaged
    }



In [25]:
import torch
import torch.nn.functional as F
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import TruncatedSVD
import pandas as pd
from scipy import sparse


def recommend_items_for_user_torch(user_id, df, interaction_data, top_n=5):
    user_profile = create_weighted_user_profile(user_id, df, interaction_data)
    vectorizer = TfidfVectorizer(stop_words='english')
    all_profiles = df['category'].tolist() + [user_profile]
    profile_matrix = vectorizer.fit_transform(all_profiles)
    cosine_sim = cosine_similarity(profile_matrix[-1:], profile_matrix[:-1])
    sim_scores = list(enumerate(cosine_sim[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    item_indices = [i[0] for i in sim_scores[:top_n]]
    recommended_items = df.iloc[item_indices][['item_id', 'category']]
    return recommended_items['item_id'].tolist()



In [26]:

def create_weighted_user_profile(user_id, df, interaction_data):
    if user_id not in interaction_data.index:
        return ''
    user_interactions = interaction_data.loc[user_id]
    positive_items = user_interactions[user_interactions > 0].index.tolist()
    user_items_df = df[df['item_id'].isin(positive_items)]
    weighted_categories = user_items_df.groupby('category')['interaction_strength'].sum()
    user_profile = ' '.join([f"{category} " * int(round(weight)) for category, weight in weighted_categories.items()])
    return user_profile.strip()

def recommend_items_for_user_torch(user_id, df, interaction_data, top_n=5):
    user_profile = create_weighted_user_profile(user_id, df, interaction_data)
    vectorizer = TfidfVectorizer(stop_words='english')
    all_categories = df['category'].tolist() + [user_profile]
    tfidf_matrix = vectorizer.fit_transform(all_categories)
    user_vec = tfidf_matrix[-1]
    item_vecs = tfidf_matrix[:-1]
    cosine_sim = cosine_similarity(user_vec, item_vecs).flatten()
    top_indices = cosine_sim.argsort()[::-1][:top_n]
    recommended_items = df.iloc[top_indices]['item_id'].tolist()
    return recommended_items

def calculate_metrics_for_user(user_id, actual_item, recommended_items, engagement_group, k=5):
    hr = hr_at_k([actual_item], recommended_items, k)
    mrr = mrr_at_k([actual_item], recommended_items, k)
    relevance = [1 if item == actual_item else 0 for item in recommended_items[:k]]
    dcg = sum(rel / np.log2(idx + 2) for idx, rel in enumerate(relevance))
    idcg = sum(1 / np.log2(idx + 2) for idx in range(min(k, len(relevance))))
    ndcg = dcg / idcg if idcg > 0 else 0
    cv = coefficient_of_variance(relevance)
    return {
        'User': user_id,
        'NDCG@K': ndcg,
        'HR@K': hr,
        'MRR@K': mrr,
        'CV': cv,
        'Engagement Group': engagement_group
    }




In [27]:
def update_user_profile(user_id, accepted_item, train_matrix, item_metadata=None):
    """
    Updates a user's profile (train_matrix) by incrementing interaction with the accepted item.

    Parameters:
        user_id (int or str): ID of the user.
        accepted_item (int or str): ID of the accepted item.
        train_matrix (pd.DataFrame): User-item interaction matrix.
        item_metadata (pd.DataFrame, optional): DataFrame with item metadata, including 'item_id' and 'category'.
    Return:
        train_matrix: Updated user-item interaction matrix.
    """
    if item_metadata is not None:
        row = item_metadata[item_metadata['item_id'] == accepted_item]
        if not row.empty:
            item_category = row['category'].iloc[0]

    if user_id not in train_matrix.index:
        train_matrix.loc[user_id] = 0

    if accepted_item not in train_matrix.columns:
        train_matrix[accepted_item] = 0


    train_matrix.loc[user_id, accepted_item] += 1

    return train_matrix

In [28]:
test_df['user_profile'] = test_df['user_id'].apply(lambda user_id: create_weighted_user_profile(user_id, df_modcloth_clean, train_user_item_matrix))

In [29]:
def coef_variation(arr):
    """
    Calculates the coefficient of variation for a list of values.
    Args:
        arr: values
    Returns:
        float: Coefficient of variation
    """
    mean_val = np.mean(arr)
    if mean_val == 0:
        return 0
    return np.std(arr) / mean_val


In [ ]:
###############################################
# HYPERPARAMETER
###############################################
from sklearn.model_selection import ParameterGrid
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np


param_grid = {
    'tfidf_max_df': [0.7, 0.8, 0.9, 1.0],
    'tfidf_min_df': [0.0, 0.01, 0.05, 0.1],
    'k': [5, 10, 15]
}

grid = ParameterGrid(param_grid)

best_params = None
best_avg_ndcg = -1
results = []

def recommend_items_for_user_tuned(user_id, df, interaction_data, tfidf_vectorizer, top_n=5):
    user_profile = create_weighted_user_profile(user_id, df, interaction_data)
    all_categories = df['category'].tolist() + [user_profile]
    tfidf_matrix = tfidf_vectorizer.fit_transform(all_categories)
    user_vec = tfidf_matrix[-1]
    item_vecs = tfidf_matrix[:-1]
    cosine_sim = cosine_similarity(user_vec, item_vecs).flatten()
    top_indices = cosine_sim.argsort()[::-1][:top_n]
    recommended_items = df.iloc[top_indices]['item_id'].tolist()
    return recommended_items


for params in grid:
    print(f"Testing parameters: {params}")


    tfidf_vectorizer = TfidfVectorizer(stop_words='english',
                                       max_df=params['tfidf_max_df'],
                                       min_df=params['tfidf_min_df'])

    round_metrics = []
    test_items = df_modcloth_clean.groupby("user_id")["item_id"].first().to_dict()

    sample_users = list(test_items.keys())

    for user_id in sample_users:
        actual_item = test_items[user_id]
        if user_id not in train_user_item_matrix.index:
             continue

        user_engagement = df_modcloth_clean[df_modcloth_clean["user_id"] == user_id]["user_activity"].iloc[0]

        recommended_items = recommend_items_for_user_tuned(user_id, df_modcloth_clean, train_user_item_matrix, tfidf_vectorizer, top_n=params['k'])
        metrics = calculate_metrics_for_user(user_id, actual_item, recommended_items, user_engagement, k=params['k'])
        round_metrics.append(metrics)

    round_metrics_df = pd.DataFrame(round_metrics)
    if not round_metrics_df.empty:
        avg_ndcg = round_metrics_df['NDCG@K'].mean()
        avg_hr = round_metrics_df['HR@K'].mean()
        avg_mrr = round_metrics_df['MRR@K'].mean()
    else:
        avg_ndcg, avg_hr, avg_mrr = 0, 0, 0

    print(f"Average NDCG: {avg_ndcg}, Average HR: {avg_hr}, Average MRR: {avg_mrr}")


    results.append({
        'params': params,
        'avg_ndcg': avg_ndcg,
        'avg_hr': avg_hr,
        'avg_mrr': avg_mrr
    })

    if avg_ndcg > best_avg_ndcg:
        best_avg_ndcg = avg_ndcg
        best_params = params

print("\n--- Tuning Complete ---")
print(f"Best parameters found: {best_params}")
print(f"Best average NDCG: {best_avg_ndcg}")
results_df = pd.DataFrame(results)
print("\nResults for all parameter combinations:")
print(results_df)
# Testing parameters: {'k': 5, 'tfidf_max_df': 0.9, 'tfidf_min_df': 0.1}

Testing parameters: {'k': 5, 'tfidf_max_df': 0.7, 'tfidf_min_df': 0.0}
Average NDCG: 0.019547912628855826, Average HR: 0.02049893637594276, Average MRR: 0.01995315756677195
Testing parameters: {'k': 5, 'tfidf_max_df': 0.7, 'tfidf_min_df': 0.01}
Average NDCG: 0.01808875400981178, Average HR: 0.018887384774060465, Average MRR: 0.018416811706310834
Testing parameters: {'k': 5, 'tfidf_max_df': 0.7, 'tfidf_min_df': 0.05}
Average NDCG: 0.01912166265113158, Average HR: 0.020047701927415715, Average MRR: 0.019566385182320203
Testing parameters: {'k': 5, 'tfidf_max_df': 0.7, 'tfidf_min_df': 0.1}
Average NDCG: 0.01912166265113158, Average HR: 0.020047701927415715, Average MRR: 0.019566385182320203
Testing parameters: {'k': 5, 'tfidf_max_df': 0.8, 'tfidf_min_df': 0.0}
Average NDCG: 0.019547912628855826, Average HR: 0.02049893637594276, Average MRR: 0.01995315756677195
Testing parameters: {'k': 5, 'tfidf_max_df': 0.8, 'tfidf_min_df': 0.01}
Average NDCG: 0.01808875400981178, Average HR: 0.018887384

In [ ]:
def recommend_items_for_user_torch(user_id, df, interaction_data, top_n=5):
    user_profile = create_weighted_user_profile(user_id, df, interaction_data)
    fidf_vectorizer = TfidfVectorizer(stop_words='english',
                                       max_df=0.9,
                                       min_df=0.1)
    all_categories = df['category'].tolist() + [user_profile]
    tfidf_matrix = vectorizer.fit_transform(all_categories)
    user_vec = tfidf_matrix[-1]
    item_vecs = tfidf_matrix[:-1]
    cosine_sim = cosine_similarity(user_vec, item_vecs).flatten()
    top_indices = cosine_sim.argsort()[::-1][:top_n]
    recommended_items = df.iloc[top_indices]['item_id'].tolist()
    return recommended_items

In [ ]:
###############################################
# 3. Multiple rounds
###############################################

def run_rounds(df, train_matrix, rounds=3):
    """Runs multiple rounds of recommendations and updates user profiles.
    Args:
    df: dataset
    train_matrix: user-item interaction matrix
    rounds: number of rounds to run
    :return: list of all round metrics

    """

    all_round_metrics = []

    for round_num in range(rounds):
        print(f"\n--- Round {round_num + 1} ---")
        round_metrics = []
        test_items = df.groupby("user_id")["item_id"].first().to_dict()


        for user_id, actual_item in test_items.items():
            if user_id not in train_matrix.index:
                continue
            user_engagement = df[df["user_id"] == user_id]["user_activity"].iloc[0]
            recommended_items = recommend_items_for_user_torch(user_id, df, train_matrix)
            metrics = calculate_metrics_for_user(user_id, actual_item, recommended_items, user_engagement)
            round_metrics.append(metrics)

            if round_num < rounds :
                train_matrix = update_user_profile(user_id, actual_item, train_matrix, df)

        round_metrics_df = pd.DataFrame(round_metrics)

        def get_ucv(metric_name):
            low = round_metrics_df[round_metrics_df['Engagement Group'] == 'Low'][metric_name].values
            high = round_metrics_df[round_metrics_df['Engagement Group'] != 'Low'][metric_name].values
            return (coef_variation(low) + coef_variation(high)) / 2

        ucv_ndcg = get_ucv('NDCG@K')
        ucv_hr = get_ucv('HR@K')
        ucv_mrr = get_ucv('MRR@K')
        ucv_cv = get_ucv('CV')
        avg_ndcg = round_metrics_df['NDCG@K'].mean()
        avg_hr = round_metrics_df['HR@K'].mean()
        avg_mrr = round_metrics_df['MRR@K'].mean()
        print(f"Average NDCG: {avg_ndcg}")
        print(f"Average HR: {avg_hr}")
        print(f"Average MRR: {avg_mrr}")

        print(f"UCV@K for NDCG in round {round_num + 1}: {ucv_ndcg:.6f}")
        print(f"UCV@K for HR in round {round_num + 1}: {ucv_hr:.6f}")
        print(f"UCV@K for MRR in round {round_num + 1}: {ucv_mrr:.6f}")


        grouped_metrics = round_metrics_df.groupby("Engagement Group").agg(
            {"NDCG@K": "mean", "HR@K": "mean", "MRR@K": "mean", "CV": "mean"}
        )
        grouped_metrics['UCV_NDCG'] = ucv_ndcg
        all_round_metrics.append(grouped_metrics)

        print(grouped_metrics)

    return all_round_metrics


all_round_metrics = run_rounds(df_modcloth_clean, train_user_item_matrix)
print(f"Overall Results: {all_round_metrics}")



--- Round 1 ---
Average NDCG: 0.02261910069964853
Average HR: 0.02436666022046026
Average MRR: 0.022874363437117257
UCV@K for NDCG in round 1: 6.980394
UCV@K for HR in round 1: 6.667746
UCV@K for MRR in round 1: 6.925614
                    NDCG@K      HR@K     MRR@K        CV  UCV_NDCG
Engagement Group                                                  
High              0.012574  0.015217  0.012983  0.005629  6.980394
Low               0.033735  0.034492  0.033820  0.001666  6.980394

--- Round 2 ---
Average NDCG: 0.0238132488534582
Average HR: 0.025784825630116675
Average MRR: 0.024086250241732738
UCV@K for NDCG in round 2: 6.733633
UCV@K for HR in round 2: 6.421744
UCV@K for MRR in round 2: 6.679043
                    NDCG@K      HR@K     MRR@K        CV  UCV_NDCG
Engagement Group                                                  
High              0.013850  0.016812  0.014284  0.006365  6.733633
Low               0.034839  0.035714  0.034933  0.001937  6.733633

--- Round 3 ---
Ave

In [ ]:


def calculate_di_gru(all_round_metrics):
    """
    Calculates Disparate Impact and Group Recommender Unfairness for NDCG and HR
    for each round of metrics.

    Args:
        all_round_metrics (list): A list of DataFrames, where each DataFrame contains
                                   metrics for a single round, grouped by Engagement Group.
                                   Each DataFrame is expected to have 'NDCG@K' and 'HR@K' columns
                                   and 'Low' and 'High' index labels.
    """
    for round_num, round_metrics_df in enumerate(all_round_metrics):
        print(f"\n--- Fairness Metrics for Round {round_num + 1} ---")


        if 'Low' in round_metrics_df.index and 'High' in round_metrics_df.index:

            ndcg_low = round_metrics_df.loc["Low", "NDCG@K"]
            ndcg_high = round_metrics_df.loc["High", "NDCG@K"]
            hr_low = round_metrics_df.loc["Low", "HR@K"]
            hr_high = round_metrics_df.loc["High", "HR@K"]
            mrr_low = round_metrics_df.loc["Low", "MRR@K"]
            mrr_high = round_metrics_df.loc["High", "MRR@K"]

            di_ndcg = calculate_disparate_impact([ndcg_low], [ndcg_high])
            di_hr = calculate_disparate_impact([hr_low], [hr_high])
            di_mrr = calculate_disparate_impact([mrr_low], [mrr_high])

            gru_ndcg = calculate_group_recommender_unfairness([ndcg_low], [ndcg_high])
            gru_hr = calculate_group_recommender_unfairness([hr_low], [hr_high])
            gru_mrr = calculate_group_recommender_unfairness([mrr_low], [mrr_high])


            print(f"NDCG:")
            print(f"  Disparate Impact (Low/High): {di_ndcg:.4f}")
            print(f"  Group Recommender Unfairness (Abs Diff): {gru_ndcg:.4f}")
            print(f"MRR:")
            print(f"  Disparate Impact (Low/High): {di_mrr:.4f}")
            print(f"  Group Recommender Unfairness (Abs Diff): {gru_mrr:.4f}")

            print(f"HR@K:")
            print(f"  Disparate Impact (Low/High): {di_hr:.4f}")
            print(f"  Group Recommender Unfairness (Abs Diff): {gru_hr:.4f}")
        else:
            print("Could not calculate fairness metrics as one or both engagement groups are missing.")

calculate_di_gru(all_round_metrics)


--- Fairness Metrics for Round 1 ---
NDCG:
  Disparate Impact (Low/High): 2.6509
  Group Recommender Unfairness (Abs Diff): 0.0181
MRR:
  Disparate Impact (Low/High): 2.5490
  Group Recommender Unfairness (Abs Diff): 0.0178
HR@K:
  Disparate Impact (Low/High): 2.4479
  Group Recommender Unfairness (Abs Diff): 0.0176

--- Fairness Metrics for Round 2 ---
NDCG:
  Disparate Impact (Low/High): 2.5014
  Group Recommender Unfairness (Abs Diff): 0.0179
MRR:
  Disparate Impact (Low/High): 2.3965
  Group Recommender Unfairness (Abs Diff): 0.0175
HR@K:
  Disparate Impact (Low/High): 2.2843
  Group Recommender Unfairness (Abs Diff): 0.0172

--- Fairness Metrics for Round 3 ---
NDCG:
  Disparate Impact (Low/High): 2.5014
  Group Recommender Unfairness (Abs Diff): 0.0179
MRR:
  Disparate Impact (Low/High): 2.3965
  Group Recommender Unfairness (Abs Diff): 0.0175
HR@K:
  Disparate Impact (Low/High): 2.2843
  Group Recommender Unfairness (Abs Diff): 0.0172
